In [14]:
#평가 스케일
def rmsle(y, y_,convertExp=True):
    if convertExp:
        y = np.exp(y), 
        y_ = np.exp(y_)
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))

from sklearn import metrics
rmsle_scorer = metrics.make_scorer(rmsle)

# 전처리

In [15]:
import pandas as pd
import numpy as np

In [16]:
df = pd.read_csv("/Users/yoonjeong/Desktop/bf_covid_train.csv")
df2 = pd.read_csv("/Users/yoonjeong/Desktop/bf_covid_test.csv")

In [17]:
bf_train = df.drop(['clustering', 'Unnamed: 0'], axis=1)
bf_test = df2.drop(['clustering', 'Unnamed: 0'], axis=1)

모델링

In [18]:
# X : features, y: target variable
X_train=  bf_train[bf_train.columns.difference(["이용자 수"])]
X_test= bf_test[bf_test.columns.difference(["이용자 수"])]
y_train= bf_train["이용자 수"]
y_test= bf_test['이용자 수']

# random forest

In [19]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 10, stop = 30, num = 5)]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(10, 50, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [10, 15, 20, 25, 30], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, None], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]}


In [20]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
grid_rf = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=2, random_state=42, n_jobs = -1,  scoring=rmsle_scorer)
grid_rf.fit(X_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


/Users/yoonjeong/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/yoonjeong/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf]
  warnings.warn(
/Users/yoonjeong/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:925: RuntimeWarning: invalid value encountered in subtra

RandomizedSearchCV(cv=10, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2, 5],
                                        'n_estimators': [10, 15, 20, 25, 30]},
                   random_state=42, scoring=make_scorer(rmsle), verbose=2)

In [35]:
from sklearn.model_selection import GridSearchCV
rmsle_scorer = metrics.make_scorer(rmsle,greater_is_better=False)
param_dict = {"criterion" : ['gini','entropy'],"max_depth": range(1,10)}
grid_rf = GridSearchCV(rf,param_dict, scoring=rmsle_scorer, cv=5 , verbose=1, n_jobs=-1)
grid_rf.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


NotFittedError: All estimators failed to fit

In [21]:
pred = grid_rf.predict(X_train)
print('RMSLE Value For Random Forest_bf_train: %.4f' % (rmsle(pred,y_train,False)))
print("\n")


y_pred = grid_rf.predict(X_test)
print('RMSLE Value For Random Forest_bf_test : %.4f' % (rmsle(y_pred,y_test,False)))
print("\n")

RMSLE Value For Random Forest_bf_train: 0.1108


RMSLE Value For Random Forest_bf_test : 0.1571




In [22]:
import pandas as pd
rf1 = grid_rf.best_estimator_.fit(X_train, y_train)
feature_imp = pd.DataFrame(rf1.feature_importances_,index=X_train.columns, columns=['importance']).sort_values('importance',ascending=False)
feature_imp

,importance
버스정류장,0.168983
샌드위치,0.131690
스타벅스,0.100360
휴일,0.089453
인구수,0.068549
출구,0.059095
은행,0.057438
영화관_b,0.053800
노령화지수,0.043460
올리브영,0.027358
